In [2]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def video_to_frames(video_path, output_folder=None, return_frames=True):
    """
    Converts a video file to an array of image frames.

    Parameters:
    - video_path: Path to the input video file
    - output_folder: Optional folder to save the extracted frames as image files
    - return_frames: Whether to return the frames as a numpy array

    Returns:
    - frames: List of numpy arrays representing each frame (if return_frames=True)
    """

    if not os.path.isfile(video_path):
        raise FileNotFoundError(f"Video file not found: {video_path}")

    if output_folder is not None and not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise Exception(f"Error opening video file: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video properties: {width}x{height}, {fps} fps, {frame_count} frames")

    frames = [] if return_frames else None

    for i in tqdm(range(frame_count), desc="Processing frames"):
        ret, frame = cap.read()

        if not ret:
            print(f"Warning: Could only read {i} frames out of {frame_count}")
            break

        if return_frames:
            frames.append(frame)

        if output_folder is not None:
            frame_filename = os.path.join(output_folder, f"frame_{i:06d}.jpg")
            cv2.imwrite(frame_filename, frame)

    cap.release()

    return frames

def frames_to_video(frames, output_path, fps=30, codec='mp4v'):
    """
    Converts an array of image frames to a video file.

    Parameters:
    - frames: List or array of numpy arrays representing each frame
    - output_path: Path to save the output video
    - fps: Frames per second for the output video
    - codec: FourCC codec code (default: mp4v for .mp4 files)

    Returns:
    - output_path: Path to the saved video file
    """
    if not frames:
        raise ValueError("No frames provided")

    height, width = frames[0].shape[:2]

    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame in tqdm(frames, desc="Writing video"):
        out.write(frame)

    out.release()

    print(f"Video saved to {output_path}")
    return output_path

# function to create video from a folder of images
def images_to_video(image_folder, output_path, fps=30, codec='mp4v', pattern="frame_*.jpg"):
    """
    Converts a folder of images to a video file.

    Parameters:
    - image_folder: Path to the folder containing image files
    - output_path: Path to save the output video
    - fps: Frames per second for the output video
    - codec: FourCC codec code (default: mp4v for .mp4 files)
    - pattern: Glob pattern to match image files

    Returns:
    - output_path: Path to the saved video file
    """
    import glob

    image_files = sorted(glob.glob(os.path.join(image_folder, pattern)))

    if not image_files:
        raise ValueError(f"No images found in {image_folder} matching pattern {pattern}")

    first_image = cv2.imread(image_files[0])
    height, width = first_image.shape[:2]

    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for image_file in tqdm(image_files, desc="Writing video"):
        frame = cv2.imread(image_file)
        out.write(frame)

    out.release()

    print(f"Video saved to {output_path}")
    return output_path

if __name__ == "__main__":
    # Convert video to frames and back
    video_path = "/content/vecteezy_the-footage-animation-of-countdown-timer-from-5-seconds_5148996 (1).mp4"
    output_frames_folder = "sample_data"
    output_video_path = "output_video.mp4"

    # Extract frames from video
    frames = video_to_frames(video_path, output_folder=output_frames_folder)

    # Convert frames back to video
    # frames_to_video(frames, output_video_path, fps=30)

    # Example 2: Create video from folder of images
    # images_to_video(output_frames_folder, "output_from_images.mp4", fps=30)

Video properties: 1920x1080, 30.301366855135313 fps, 184 frames


Processing frames: 100%|██████████| 184/184 [00:03<00:00, 52.24it/s]
